In [9]:
from dataWrangler import *
from scrape_craigslist import Scrape
from predictions import *
import urllib.request
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import requests
import datetime
import math
import glob   
import os
import io
import re

In [10]:
li=[]
for filename in glob.glob(r"..\UsedCar-PriceEstimator\data\*.csv"):
    collection = pd.read_csv(filename, index_col=None, header=0)
    li.append(collection)
frame = pd.concat(li, axis=0, ignore_index=True)

In [11]:
lis=[]
for x in range(3,29,6):
    lis.append(singlePredict('https://sfbay.craigslist.org/eby/cto/d/newark-2006-audi-a3/7274749218.html',frame,x))

x=0
for l in lis:
    x+=(l[1])
res=int(x/len(lis))
result=((math.floor(res / 100.0)) * 100)    
    

result,lis  

audi a3 Accuracy: 98.05 %.
audi a3 Accuracy: 98.5 %.
audi a3 Accuracy: 97.55 %.
audi a3 Accuracy: 96.44 %.
audi a3 Accuracy: 96.77 %.


(5400,
 [('predicted:', 5738, 'actual:', 5900),
  ('predicted:', 4813, 'actual:', 5900),
  ('predicted:', 5489, 'actual:', 5900),
  ('predicted:', 5489, 'actual:', 5900),
  ('predicted:', 5514, 'actual:', 5900)])